### 数据集介绍
本次常规赛提供的金标准由中山大学中山眼科中心的7名眼科医生手工进行视盘像素级标注，之后由另一位高级专家将它们融合为最终的标注结果。存储为BMP图像，与对应的眼底图像大小相同，标签为0代表视盘（黑色区域）；标签为255代表其他（白色区域）。

训练数据集

文件名称：Train
Train文件夹里有fundus_images文件夹和Disc_Masks文件夹。

fundus_images文件夹内包含800张眼底彩照，分辨率为1444×1444，或2124×2056。命名形如H0001.jpg、N0001.jpg、P0001.jpg和V0001.jpg。

Disc_Masks文件夹内包含fundus_images里眼底彩照的视盘分割金标准，大小与对应的眼底彩照一致。命名前缀和对应的fundus_images文件夹里的图像命名一致，后缀为bmp。

测试数据集

文件名称：PALM-Testing400-Images

包含400张眼底彩照，命名形如T0001.jpg。

### 比赛思路
参考五月第一名代码，后面更改使用其它模型后没有超过第一名的分数，所以后面再次提交的分数均还是五月份第一名的代码所得分数

使用DeepLabv3 分数在0.93（参数与使用Unet时基本相同）

使用DeepLabv3P 分数在0.94（参数与使用DeepLabv3相同）

![](https://ai-studio-static-online.cdn.bcebos.com/e4c5bdd28975488e9400aa8842f6513167982f1989884b1b85c2575b4123c007)




下展示的是

使用Unet进行分割。对预测结果处理，如出现多个不连通区域，则只保留最大的面积那块。

In [1]:
#解压数据
!unzip -o data/data86770/seg.zip -d /home/aistudio/work

In [1]:
!git clone https://gitee.com/PaddlePaddle/PaddleSeg

## 生成train.txt 和val.txt

In [2]:
import random
import os
random.seed(2020)
mask_dir  = '/home/aistudio/work/seg/Train/masks'
img_dir = '/home/aistudio/work/seg/Train/fundus_image'
path_list = list()
for img in os.listdir(img_dir):
    img_path = os.path.join(img_dir,img)
    mask_path = os.path.join(mask_dir,img.replace('jpg', 'png'))
    path_list.append((img_path, mask_path))
random.shuffle(path_list)
ratio = 0.8
train_f = open('/home/aistudio/work/seg/Train/train.txt','w') 
val_f = open('/home/aistudio/work/seg/Train/val.txt' ,'w')

for i ,content in enumerate(path_list):
    img, mask = content
    text = img + ' ' + mask + '\n'
    if i < len(path_list) * ratio:
        train_f.write(text)
    else:
        val_f.write(text)
train_f.close()
val_f.close()

### 配置文件如下
```
batch_size: 8 #0.94969
iters: 4000

train_dataset:
  type: Dataset
  dataset_root: /home/aistudio/work/seg/Train/
  train_path: /home/aistudio/work/seg/Train/train.txt
  num_classes: 2
  transforms:
    - type: Resize
      target_size: [512, 512]
    # - type: RandomRotation
    #   max_rotation: 15
    - type: RandomHorizontalFlip
    - type: RandomDistort
      brightness_range: 0.4
      contrast_range: 0.4
      saturation_range: 0.4
    - type: Normalize
  mode: train

val_dataset:
  type: Dataset
  dataset_root: /home/aistudio/work/seg/Train/
  val_path: /home/aistudio/work/seg/Train/val.txt
  num_classes: 2
  transforms:
    - type: Resize
      target_size: [512, 512]
    - type: Normalize
  mode: val


optimizer:
  type: sgd
  momentum: 0.9
  weight_decay: 4.0e-5

learning_rate:
  value: 0.00125
  decay:
    type: poly
    power: 0.9
    end_lr: 0.0

loss:
  types:
    - type: MixedLoss
      losses:
        - type: CrossEntropyLoss
        - type: DiceLoss
      coef: [0.7, 0.3]
  coef: [1]

model:
  type: UNet
  num_classes: 2
  use_deconv: False
  pretrained: /home/aistudio/unetmodel.pdparams

```

### 开始训练

In [3]:
%cd /home/aistudio/PaddleSeg
!python train.py \
       --config configs/unet_PALM.yml \
       --do_eval \
       --use_vdl \
       --save_interval 100 \
       --save_dir output_unet_PALMoutput

### 验证

In [4]:
%cd /home/aistudio/PaddleSeg
!python val.py --config configs/unet_PALM.yml  --model_path output_unet_PALMoutput/best_model/model.pdparams 

### 预测

In [5]:
%cd /home/aistudio/PaddleSeg
!python predict.py \
       --config configs/unet_PALM.yml \
       --model_path output_unet_PALMoutput/best_model/model.pdparams \
       --image_path /home/aistudio/work/seg/test \
       --save_dir output_unet_PALMoutput/result

### 生成结果

In [6]:
import os 
import cv2
result_path = '/home/aistudio/PaddleSeg/output_unet_PALMoutput/result/pseudo_color_prediction'
dist_path = '/home/aistudio/Disc_Segmentation'
for img_name in os.listdir(result_path):
    img_path = os.path.join(result_path, img_name)
    img = cv2.imread(img_path)
    g  = img[:,:,1]
    ret, result = cv2.threshold(g, 127,255, cv2.THRESH_BINARY_INV)
    cv2.imwrite(os.path.join(dist_path,img_name), result)


### 假如预测中出现多个不连通的区域，只保留最大的区域

In [7]:
import os 
import cv2
import matplotlib.pyplot as plt
def cnt_area(cnt):
    area = cv2.contourArea(cnt)
    return area

result_path = '/home/aistudio/PaddleSeg/output_unet_PALMoutput/result/pseudo_color_prediction'
dist_path = '/home/aistudio/Disc_Segmentation'
for img_name in os.listdir(result_path):
    img_path = os.path.join(result_path, img_name)
    img = cv2.imread(img_path)
    g  = img[:,:,1]
    ret, threshold = cv2.threshold(g, 127,255, cv2.THRESH_BINARY)


    contours, hierarch = cv2.findContours(threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    contours.sort(key=cnt_area, reverse=True)
    if len(contours) > 1:
        for i in range(1,len(contours)):
            cv2.drawContours(threshold, [contours[i]], 0, 0, -1)
    _,result = cv2.threshold(threshold, 127, 255, cv2.THRESH_BINARY_INV)
    cv2.imwrite(os.path.join(dist_path, img_name), result)


In [8]:
%cd /home/aistudio/Disc_Segmentation
!zip -r -oq /home/aistudio/Disc_Segmentation.zip ./

/home/aistudio/Disc_Segmentation


### 总结
模型构建思路及调优过程（可具体包括：思路框架图、思路步骤详述、模型应用+调优过程）

【模型】Unet

【数据增强】图片大小512x512，水平翻转，对比度随机改变等数据增强

【对预测结果进行处理】假如预测中出现多个不连通的区域，只保留最大的区域
【提高】在原来使用Unet的基础上，尝试了Deeplab和Deeplab+ 网络，可能因为参数设置问题（经过多次尝试），均没有得到更高分数
		后在原来基础上在batch_size: 8  iters: 4000  得分0.94969 限于时间原因没有增加iters  如果增加iters 可能超过原来的分数